In [0]:
%run /Workspace/Users/amank0639@gmail.com/fmcg_repo/consolidated_pipeline/1_Setup/utilities

In [0]:
dbutils.widgets.text("catalog","test_catalog")
dbutils.widgets.text("Data Source","test_source")

In [0]:
catalog=dbutils.widgets.get("catalog")
data_source=dbutils.widgets.get("Data Source")

base_path=f"s3://sportsbar-bucket/{data_source}/*.csv"

In [0]:
from pyspark.sql.types import StructType,StructField,StringType
from pyspark.sql.functions import col,trim,current_timestamp
customer_schema=(
    StructType([
        StructField("customer_id",StringType(),True),
        StructField("customer_name",StringType(),True),
        StructField("city",StringType(),True)
    ])
    )
df=(spark.
    read.
    format("csv").
    option("header",True).
    schema(customer_schema).
    load(base_path).
    withColumn("read_timestamp",current_timestamp()).
    select('*','_metadata.file_name','_metadata.file_size')
    )
display(df)

In [0]:
df\
.write\
.mode("overwrite")\
.format("delta")\
.option("enableChangeDataFeed","true")\
.saveAsTable(f"{catalog}.{bronze_schema}.{data_source}")